# Using the Python SDK for autoRIFT

HyP3's Python SDK `hyp3_sdk` provides a convenience wrapper around the HyP3 API and HyP3 jobs.


The HyP3 SDK can be installed using [Anaconda/Miniconda (recommended)](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
via [`conda`](https://anaconda.org/conda-forge/hyp3_sdk):

```
conda install -c conda-forge hyp3_sdk
```

Or using [`pip`](https://pypi.org/project/hyp3-sdk/):

```
python -m pip install hyp3_sdk
```

Full documentation of the SDK can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [ ]:
# initial setup
import hyp3_sdk as sdk

AUTORIFT_API = 'https://hyp3-autorift.asf.alaska.edu/'

## Authenticating to the API

The SDK will pull your [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) credentials out of `~/.netrc` if they
exist by default, or you can pass your credentials in directly

In [ ]:
# .netrc
hyp3 = sdk.HyP3(AUTORIFT_API)

In [ ]:
# or enter your credentials
from getpass import getpass
username = 'MY_EDL_USERNAME'
password = getpass()  # will prompt for a password
hyp3 = sdk.HyP3(AUTORIFT_API, username=username, password=password)

## Submitting jobs

AutoRIFT jobs can be submitted using the `hyp3.submit_autorift_job()` method.

### Sentinel-1

Sentinel-1 jobs are submitted using the [ESA granule ID](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/naming-conventions)

In [ ]:
s1_pairs = [
    ('S1A_IW_SLC__1SSH_20170221T204710_20170221T204737_015387_0193F6_AB07',
     'S1B_IW_SLC__1SSH_20170227T204628_20170227T204655_004491_007D11_6654'),
    ('S1B_IW_SLC__1SDH_20180821T204618_20180821T204645_012366_016CC2_59A7',
     'S1B_IW_SLC__1SDH_20180809T204617_20180809T204644_012191_01674F_9345'),
    ('S1A_IW_SLC__1SSH_20151214T080202_20151214T080229_009035_00CF68_780C',
     'S1A_IW_SLC__1SSH_20151120T080202_20151120T080229_008685_00C5A7_105E'),
    ('S1A_IW_SLC__1SSH_20150909T162413_20150909T162443_007640_00A97D_922B',
     'S1A_IW_SLC__1SSH_20150828T162412_20150828T162431_007465_00A4AF_DC3E'),
]

s1_jobs = sdk.Batch()
for g1, g2 in s1_pairs:
    s1_jobs += hyp3.submit_autorift_job(g1, g2, name='s1-example')

Here we've given each job the name `s1-example`, which we can use later to search for these jobs.

### Sentinel-2

Seninel-2 jobs can be submitted using either the [ESA granule ID](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention)
or the [COG ID on AWS](https://registry.opendata.aws/sentinel-2-l2a-cogs/#:~:text=The%20Sentinel%2D2%20mission%20is,great%20use%20in%20ongoing%20studies.)

In [ ]:
s2_pairs = [
    # Can be either ESA granule IDs
    ('S2B_MSIL2A_20201016T161349_N0214_R111_T07CDL_20201016T195625',
     'S2B_MSIL2A_20201030T155329_N0214_R025_T07CDL_20201030T200159'),
    # or AWS COG IDs
    ('S2B_22WEB_20200903_0_L2A', 'S2B_22WEB_20200913_0_L2A'),
]

s2_jobs = sdk.Batch()
for g1, g2 in s2_pairs:
    s2_jobs += hyp3.submit_autorift_job(g1, g2, name='s2-example')


### Landsat 8

**NOTE: Landsat support is coming soon**

In [ ]:
# FIXME: Currently unsupported
l8_pairs = []

l8_jobs = sdk.Batch()
for g1, g2 in l8_pairs:
    l8_jobs += hyp3.submit_autorift_job(g1, g2, name='l8-example')

## Monitoring jobs

One jobs are submitted, you can either watch the jobs until they finish. E.g.,

In [ ]:
s1_jobs = hyp3.watch(s1_jobs)

which will require you to keep the cell/terminal running. Or, you can come back later and search for jobs. E.g.,

In [ ]:
s1_jobs = hyp3.find_jobs(name='s1-example')
s1_jobs = hyp3.watch(s1_jobs)

### Downloading files

Batches are collections of jobs, and they provide a snapshot of the job status when the job was created or last
refreshed. To get updated information on a batch

In [ ]:
s1_jobs = hyp3.refresh(s1_jobs)
s1_jobs.complete()

`hyp3.watch()` will return a refreshed batch once the batch has completed.

Batches can be added together

In [ ]:
print(f'Number of Jobs:\n  S1:{len(s1_jobs)}\n  S2:{len(s2_jobs)}\n  L8:{len(l8_jobs)}')
all_jobs = s1_jobs + s2_jobs + l8_jobs
print(f'Total number of Jobs: {len(all_jobs)}')

You can check if every job was successful

In [ ]:
all_jobs.succeeded()

and filter jobs by status

In [ ]:
succeeded_jobs = all_jobs.filter_jobs(succeeded=True, running=False, failed=False)
failed_jobs = all_jobs.filter_jobs(succeeded=False, running=False, failed=True)

You can download the files for all successful jobs

In [ ]:
file_list = succeeded_jobs.download_files('./')

*Note: only succeeded jobs will have files to download.*